# Citing and reporting literature values

Table of Contents
* [Citing many data sources](#citing)
* [Create table of asteroid parameters using rocks](#create-table)

This notebook shows how `rocks` simplifies the discovery and proper attribution of literature parameter values.
As always:

In [1]:
import rocks

## Citing data sources <a class="anchor" id="citing"></a>

Let's assume we plotted a histogram of the diameters the first 1000 numbered asteroids here. We want to cite the data sources in the appendix.

In [2]:
targets = range(1, 1000)
targets = rocks.rocks(targets) # get the ssocards of the first 1,000 asteroids

All we have to do is iterate over the targets and, if they have a diameter entry in their ssoCard, note the bibcodes of the given values. There may be multiple bibcodes per asteroid in the case where the best-estimate diameter is a mean value. This is why the objects returned by the `bibref` attribute are always lists.

In [3]:
bibcodes = [] # to store the results

for target in targets:
    
    if not target.diameter:  # if no diameter is available
        continue             # we don't care about this one
        
    bibcodes += target.diameter.bibref.bibcode  # `bibref.bibcode` is a list. += adds two lists in python

And that's already it. We can look at who contributed the most diameters using the collection module.

In [4]:
# we use a 'set' to remove duplicate entries
bibcodes_for_latex = ','.join(set(bibcodes))

# triple {{{}}} because python f-strings require this
print(f'This study uses data from the following works: \citep{{{bibcodes_for_latex}}}')

This study uses data from the following works: \citep{2020A&A...638A..84A,2011ApJ...741...68M,2018Icar..309..134P,2008Icar..195..220S,2011Sci...334..487S,2023ApJ...944..202J,1999Icar..140...17T,2017A&A...604A..64M,2010Icar..205..460C,2019A&A...624A.121H,2016A&A...591A..14A,2020AJ....159..264J,2017MNRAS.471..941B,2023A&A...670A..52M,2017AJ....154..168M,2021A&A...654A..56V,2020A&A...638L..15F,2014ApJ...783L..37M,2020NatAs...4..136V,2010Icar..207..285O,2015ApJ...814..117N,2018Icar..309..297H,2015MNRAS.448.3382T,2020MNRAS.499.4545D,2014MNRAS.443.1802B,2007Icar..186..126M,2022A&A...662A..71F,2015MPBu...42..129T,2018A&A...618A.154V,2017A&A...603A..55A,2021A&A...653A..57M,2020A&A...633A..65H,2020A&A...641A..80Y,2017A&A...599A..36H,2012Icar..221.1130M,2013Icar..226.1045H,2018A&A...619L...3V,2014ApJ...791..121M,2018A&A...612A..85A,2022PSJ.....3...30M,2020NatAs...4..569M,2021A&A...650A.129C,2017A&A...601A.114H,2019pdss.data....3H,2012ApJ...759L...8M,2017Icar..281..388S,2009P&SS...57..259D,2014Ic

If your bibliography file uses the bibcodes as article identifiers, you can write the result straight to file in TeX format and add it your article. If you decide to add/remove asteroids from the analysis, just rerun the script to update your bibliography.

We can further use the `collections` module to sort the references by number of contributions.

In [5]:
from collections import Counter

count = Counter(bibcodes)  # count how often a given bibcode appears

for bibcode, number in count.most_common():
    print(f'{bibcode} contributed to the diameters of {number} asteroids')

2022PSJ.....3...56H contributed to the diameters of 403 asteroids
2019pdss.data....3H contributed to the diameters of 247 asteroids
2018A&A...612A..85A contributed to the diameters of 220 asteroids
2011PASJ...63.1117U contributed to the diameters of 220 asteroids
2012ApJ...759L...8M contributed to the diameters of 170 asteroids
2011ApJ...741...68M contributed to the diameters of 154 asteroids
2010AJ....140..933R contributed to the diameters of 146 asteroids
2014ApJ...791..121M contributed to the diameters of 133 asteroids
2016AJ....152...63N contributed to the diameters of 113 asteroids
2015ApJ...814..117N contributed to the diameters of 101 asteroids
2021PSJ.....2..162M contributed to the diameters of 71 asteroids
2018Icar..309..297H contributed to the diameters of 66 asteroids
2022PSJ.....3...30M contributed to the diameters of 60 asteroids
2020PSJ.....1....5M contributed to the diameters of 60 asteroids
2017AJ....154..168M contributed to the diameters of 43 asteroids
2017A&A...601A.

## Create a parameter table for a publication  <a class="anchor" id="create-table"></a>

A more advanced example. We create a TeX table comparing different parameters of Ryugu and Bennu.
This example will create the table as a single python string, in real practice, you would write it to file and add the file to your manuscript.

First, we define the targets of our study and the parameters we want to add to the table.
If we later want to add another asteroid, we can just add its name to the `targets` list.
The names of the parameters are the ones we would usually use in the dot-notation, e.g. `sso.number` -> `'number'`.
Again, we can quickly add rows with other parameters to our table by defining it this way.

In [37]:
asteroids = ['bennu', 'ryugu']
ssos = rocks.rocks(asteroids)  # get the ssoCards and ingest the data

# Add some orbital information
parameters = [
    'class_',   # the orbital class
    'moid.emb', # minimum orbit intersection distance earth-moon-barycenter
    'albedo',
    'diameter',
    'taxonomy.class_',
    'thermal_inertia'
]

Now we need a function we returns the formatted TeX string for each row we want to add.
It should accept the parameter name, a list of parameter values, and a list of bibcodes for the references column. It looks more complicated than it really is.

In [38]:
def create_row(parameter, values, bibcodes):
    """Create TeX string for table row.
    
    parameter : str
        The name of the parameter
    values : list of str
        The values to add for each asteroid to the row.
    bibcodes : list of str
        The references to cite.
    """
    if bibcodes: # only try to cite if there are actually bibcodes to cite
        cite = fr"\citet{{{','.join(bibcodes)}}}"
    else:
        cite = ''
        
    return fr"{parameter} & {' & '.join(values)} & {cite} \\"

Let's see this in action. We add the header row of the table.

In [39]:
header = create_row(parameter='', values=[f'({sso.number}) {sso.name}' for sso in ssos], bibcodes=[])

# The header needs a \topline
header = header + r'\topline'
print(header)

 & (101955) Bennu & (162173) Ryugu &  \\\topline


Now we're ready to add the actual parameters. Because we want to loop over a list of parameters,
we cannot use the usual dot-notation to get the actual parameter values. Instead, we use the `Rock.get_parameter`
helper function which accepts a parameter name as string. We further make use of the units and labels which are defined for all parameters in SsODNet.

In [48]:
rows = []  # to store the resulting rows

for parameter in parameters: # for each parameter we are interested in
    
    # values and bibcodes for this row
    values = []
    bibcodes = []
    
    for sso in ssos:         # for each asteroid
        
        # for the orbital class, we look up the value directly
        if parameter == 'class_':
            values.append(sso.class_)
            label = 'Orbital Class'
            continue
        
        # for the other parameters, we do a dynamic look-up
        param = sso.get_parameter(parameter)
        
        value = str(param)  # rocks automatically formats the parameters for you
        values.append(value)
        
        if hasattr(param, 'bibref'):  # if the parameter has a citation attached to it
            bibcodes += param.bibref.bibcode # bibref.bibcodes are always a list
        
        # we use the SsODNet label of this parameter for the table
        label = param.label
        
    row = create_row(parameter=label, values=values, bibcodes=bibcodes)
    rows.append(row)

We are ready to build the table.

In [52]:
table = fr'''
\begin{{tabular}}{{c{'c' * len(ssos)}c}}
{header}
{''.join(rows)}
\end{{tabular}}
'''
print(table.replace(r'\\', '\n'))


\begin{tabular}{cccc}
 & (101955) Bennu & (162173) Ryugu &  
\topline
Orbital Class & NEA>Apollo & NEA>Apollo &  
EMB MOID & 0.08382782 +- 0.00000000 au & 0.04965338 +- 0.00000000 au &  
Albedo & 0.044 +- 0.002  & 0.045 +- 0.002  & \citet{2019NatAs...3..341D,2019Sci...364..252S} 
Diameter & 0.488 +- 0.019 km & 0.896 +- 0.008 km & \citet{2019NatGe..12..247B,2019Sci...364..268W} 
Class & B & C &  
Thermal inertia & 401.29 +- 20.00 J.s^{-1/2}.K^{-1}.m^{-2} & 324.62 +- (65.85, -46.38) J.s^{-1/2}.K^{-1}.m^{-2} & \citet{2019NatAs...3..341D,2020Icar..34813835S,2019NatAs...3..971G,2020Natur.579..518O,2019Sci...364..252S} 

\end{tabular}



It doesn't look like much in the notebook, but the table has the parameter values we requested (including uncertainties and units) and a full bibliographic record as well. Further code could be added to format cells with missing values, but we leave the example like this for now.

$$
\begin{tabular}{cccc}
 & (101955) Bennu & (162173) Ryugu &  \\\topline
Orbital Class & NEA>Apollo & NEA>Apollo &  \\
EMB MOID & 0.08382782 +- 0.00000000 au & 0.04965338 +- 0.00000000 au &  \\
Albedo & 0.044 +- 0.002  & 0.045 +- 0.002  & \citet{2019NatAs...3..341D,2019Sci...364..252S} \\
Diameter & 0.488 +- 0.019 km & 0.896 +- 0.008 km & \citet{2019NatGe..12..247B,2019Sci...364..268W} \\
Class & B & C &  \\Thermal inertia & 401.29 +- 20.00 J.s^{-1/2}.K^{-1}.m^{-2} & 324.62 +- (65.85, -46.38) J.s^{-1/2}.K^{-1}.m^{-2} & \citet{2019NatAs...3..341D,2020Icar..34813835S,2019NatAs...3..971G,2020Natur.579..518O,2019Sci...364..252S} \\
\end{tabular}
$$